In [3]:
# Step 1: Importing Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import matplotlib.pyplot as plt
import os

In [5]:
# Step 2: Load the CSV files and paths
train_df = pd.read_csv('/Users/tanishrajput/Desktop/Projects/Diabetic  Retinopathy/Dataset/train.csv')
valid_df = pd.read_csv('/Users/tanishrajput/Desktop/Projects/Diabetic  Retinopathy/Dataset/valid.csv')
test_df = pd.read_csv('/Users/tanishrajput/Desktop/Projects/Diabetic  Retinopathy/Dataset/test.csv')

In [6]:
# Adding a column to construct the full path to images based on image_id
train_image_dir = '/Users/tanishrajput/Desktop/Projects/Diabetic  Retinopathy/Dataset/train_images/'
val_image_dir = '/Users/tanishrajput/Desktop/Projects/Diabetic  Retinopathy/Dataset/val_images/'
test_image_dir = '/Users/tanishrajput/Desktop/Projects/Diabetic  Retinopathy/Dataset/test_images/'

In [7]:
train_df['image'] = train_df['id_code'].apply(lambda x: os.path.join(train_image_dir, f"{x}.png"))
valid_df['image'] = valid_df['id_code'].apply(lambda x: os.path.join(val_image_dir, f"{x}.png"))
test_df['image'] = test_df['id_code'].apply(lambda x: os.path.join(test_image_dir, f"{x}.png"))

In [8]:
# Convert the 'diagnosis' column to string in the DataFrames
train_df['diagnosis'] = train_df['diagnosis'].astype(str)
valid_df['diagnosis'] = valid_df['diagnosis'].astype(str)
test_df['diagnosis'] = test_df['diagnosis'].astype(str)

In [9]:
# Step 3: Preprocessing the images using ImageDataGenerator

# Use ImageDataGenerator for real-time data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize the pixel values
    rotation_range=20,       # Data augmentation
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'      # Fill missing pixels after rotation/translation
)

In [10]:
# Load training images from directories and CSV
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image',
    y_col='diagnosis',    # Now this is a string
    target_size=(224, 224),  
    batch_size=32,
    class_mode='categorical' # Multiclass classification
)

Found 2930 validated image filenames belonging to 5 classes.


In [11]:
val_generator = datagen.flow_from_dataframe(
    dataframe=valid_df,
    x_col='image',
    y_col='diagnosis',    # Now this is a string
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 366 validated image filenames belonging to 5 classes.


In [12]:
# Define ImageDataGenerator for the test set (without data augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)  # Just rescale pixel values for testing

In [13]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image',
    y_col='diagnosis',    # Now this is a string
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 366 validated image filenames belonging to 5 classes.


In [14]:
# Step 4: Build the CNN Model
model = Sequential([
    # First convolutional layer
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    # Second convolutional layer
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Third convolutional layer
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Flatten and fully connected layers
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  # 5 categories for DR
])

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# Step 5: Compile the Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
# Step 6: Train the Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,  # Adjust as necessary
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator) 
)

Epoch 1/25


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-10-17 21:47:36.459175: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 7 of 8
2024-10-17 21:47:38.043067: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


92/92 ━━━━━━━━━━━━━━━━━━━━ 193s 2s/step - accuracy: 0.5044 - loss: 1.5047 - val_accuracy: 0.6721 - val_loss: 0.9452
Epoch 2/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/25


2024-10-17 21:50:37.890504: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
2024-10-17 21:50:37.930121: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-10-17 21:50:48.661988: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 6 of 8
2024-10-17 21:50:51.636704: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


92/92 ━━━━━━━━━━━━━━━━━━━━ 188s 2s/step - accuracy: 0.6894 - loss: 0.9372 - val_accuracy: 0.6858 - val_loss: 0.8438
Epoch 4/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 180us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/25


...

92/92 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.7017 - loss: 0.8537 - val_accuracy: 0.7077 - val_loss: 0.8407
Epoch 22/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 216us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 23/25


2024-10-17 22:23:42.544650: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 6 of 8
2024-10-17 22:23:45.805757: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


92/92 ━━━━━━━━━━━━━━━━━━━━ 190s 2s/step - accuracy: 0.7191 - loss: 0.8126 - val_accuracy: 0.7077 - val_loss: 0.7886
Epoch 24/25
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 218us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 25/25


2024-10-17 22:26:52.800430: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 6 of 8
2024-10-17 22:26:56.435247: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


92/92 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.6990 - loss: 0.8319 - val_accuracy: 0.7077 - val_loss: 0.7954


In [17]:
# Step 7: Evaluate on Test Set
test_loss, test_acc = model.evaluate(test_generator) 
print(f"Test Accuracy: {test_acc*100:.2f}%")

12/12 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.7264 - loss: 0.8060
Test Accuracy: 74.86%


In [18]:
# Step 8: Plot Training/Validation Accuracy & Loss
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [19]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [22]:
import tensorflow as tf
import pickle

# Load the model from the .h5 file
model = tf.keras.models.load_model('model.h5')

# Save the model to a .pkl file
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model converted and saved as model.pkl")


Model converted and saved as model.pkl


In [21]:
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
try:
    model = load_model('model.h5')
    print("Model loaded successfully.")
except OSError as e:
    print(f"Error loading model: {e}")


Model loaded successfully.
